In [ ]:
import os, sys, subprocess, time, toml
import pandas as pd
import numpy as np
from MOASMO_parameter_allbasin_emulator import allbasin_emulator_train_and_optimize, allbasin_emulator_CV_traintest_and_optimize
import run_multiple_paramsets_Derecho
from multiprocessing import Pool

iter_end = int(sys.argv[1]) # e.g., iter_end=2 means outputs from iter0 and iter1 will be used to generate new paprameters for iter 2
ncpus = int(sys.argv[2]) 
# ncpus = 20
# iterend = 1
numruns = 100

only_checkruns = False

infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
infile_param_info = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/src/parameter/CTSM_CAMELS_SA_param_240202.csv'
infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
inpath_moasmo = "/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange"
path_CTSM_case_all = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO_bigrange'

# trainmode = 'trainbasin' # allbasin; trainbasin
# trainmode = 'allbasin_2err'
# trainmode = 'spaceCV'
# trainmode = 'allbasin_50iter0'
trainmode = 'allbasin'
